In [ ]:
pip install CoolProp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.5 MB/s eta 0:00:00


In [ ]:
# ==========================================================
#  GAS TURBINE ENERGY BALANCE - SGT5-9000HL (Energy-Based)
# ==========================================================

from CoolProp.CoolProp import PropsSI

# ----------------------------------------------------------
# STEP 1: LHV OF THE NATURAL GAS MIXTURE (kJ/kg)
# ----------------------------------------------------------

composition = {
    "CH4": 89.138,
    "N2": 0.504,
    "CO2": 0.446,
    "C2H6": 5.691,
    "C3H8": 2.406,
    "nC4H10": 0.795,
    "iC4H10": 0.321,
    "nC5H12": 0.218,
    "iC5H12": 0.200,
    "nC6H14": 0.280,
    "H2S": 0.000288,
}

total = sum(composition.values())
for k in composition:
    composition[k] /= total

# LHV data (kJ/mol)
LHV_data_kJ_per_mol = {
    "CH4": 802.3,
    "C2H6": 1428.0,
    "C3H8": 2043.0,
    "nC4H10": 2658.0,
    "iC4H10": 2658.0,
    "nC5H12": 3272.0,
    "iC5H12": 3268.0,
    "nC6H14": 3890.0,
    "CO2": 0.0,
    "N2": 0.0,
    "H2S": 563.0,
}

# Molecular weights (g/mol)
MW_data = {
    "CH4": 16.043,
    "C2H6": 30.070,
    "C3H8": 44.097,
    "nC4H10": 58.124,
    "iC4H10": 58.124,
    "nC5H12": 72.151,
    "iC5H12": 72.151,
    "nC6H14": 86.178,
    "CO2": 44.010,
    "N2": 28.013,
    "H2S": 34.076,
}

# Mixture MW and LHV
MW_mix = sum(composition[comp] * MW_data[comp] for comp in composition)
LHV_mix_mol = sum(composition[comp] * LHV_data_kJ_per_mol.get(comp, 0) for comp in composition)
LHV_mix_mass = LHV_mix_mol / (MW_mix / 1000)  # kJ/kg

print("----- Natural Gas LHV Calculation -----")
print(f"LHV (kJ/kg)  = {LHV_mix_mass:.2f}")
print("-" * 55)

# ----------------------------------------------------------
# STEP 2: GAS TURBINE PERFORMANCE
# ----------------------------------------------------------

eta_GT = 0.42               # Efficiency (42%)
P_el = 593e6
Ploss = 14.59e6           # Electrical output [W]

Q_in = P_el / eta_GT        # Thermal input [W]
mfuel = Q_in / (LHV_mix_mass * 1000)  # kg/s
Q_loss = Q_in - P_el - Ploss      # Energy lost with exhaust [W]

print("----- Gas Turbine Performance -----")
print(f"Electrical Power Output = {P_el/1e6:.1f} MW")
print(f"Net Efficiency = {eta_GT*100:.1f} %")
print(f"Thermal Power Input (Q_in) = {Q_in/1e6:.2f} MW_th")
print(f"Fuel Mass Flow = {mfuel:.3f} kg/s")
print(f"Thermal Loss (Q_loss) = {Q_loss/1e6:.2f} MW")
print("-" * 55)

# ----------------------------------------------------------
# STEP 3: FLUE GAS PROPERTIES AT 670 °C
# ----------------------------------------------------------

P_ATM = 101325  # Pa
T_C = 670.0
T_K = T_C + 273.15

MOLE_FRACTIONS = {
    'CO2': 0.04925363,
    'SO2': 0.00000012,
    'N2':  0.73794838,
    'H2O': 0.10024462,
    'O2':  0.10389788,
    'Ar':  0.00865537
}

MOLAR_MASSES_KG = {
    'CO2': 44.0095 / 1000.0,
    'SO2': 64.0660 / 1000.0,
    'N2':  28.0134 / 1000.0,
    'H2O': 18.01528 / 1000.0,
    'O2':  31.9988 / 1000.0,
    'Ar':  39.948 / 1000.0,
}

MM_mix = sum(MOLE_FRACTIONS[comp] * MOLAR_MASSES_KG[comp] for comp in MOLE_FRACTIONS)

def get_cp_component(comp, T_K):
    cp_mass = PropsSI("CPMASS", "T", T_K, "P", P_ATM, comp)
    M_i = MOLAR_MASSES_KG[comp]
    cp_molar = cp_mass * M_i
    return cp_mass, cp_molar

def mixture_cp(T_K):
    cp_molar_mix = 0.0
    for comp, y_i in MOLE_FRACTIONS.items():
        _, cp_molar_i = get_cp_component(comp, T_K)
        cp_molar_mix += y_i * cp_molar_i
    cp_mass_mix = cp_molar_mix / MM_mix
    return cp_mass_mix / 1000.0  # kJ/kg/K

cp_mix = mixture_cp(T_K)

print("----- Flue Gas Properties -----")
print(f"Temperature: {T_C:.1f} °C ({T_K:.2f} K)")
print(f"Mixture Cp: {cp_mix:.3f} kJ/kg/K")
print("-" * 55)

# ----------------------------------------------------------
# STEP 4: DETERMINE EXHAUST MASS FLOW & AFR (ENERGY CONSERVATION)
# ----------------------------------------------------------

T_ref = 10 + 273.15
deltaT = T_K - T_ref

# Q_loss [W] = m_exh [kg/s] * cp_mix [kJ/kg/K]*1000 * deltaT [K]
m_exhaust = Q_loss / (cp_mix * 1000 * deltaT)  # kg/s
AFR = (m_exhaust - mfuel) / mfuel

print("----- Energy Balance and Exhaust Flow -----")
print(f"ΔT = {deltaT:.1f} K")
print(f"Exhaust Mass Flow = {m_exhaust:.2f} kg/s")
print(f"Air-Fuel Ratio (AFR) = {AFR:.2f}")
print("-" * 55)


----- Natural Gas LHV Calculation -----
LHV (kJ/kg)  = 48371.81
-------------------------------------------------------
----- Gas Turbine Performance -----
Electrical Power Output = 593.0 MW
Net Efficiency = 42.0 %
Thermal Power Input (Q_in) = 1411.90 MW_th
Fuel Mass Flow = 29.189 kg/s
Thermal Loss (Q_loss) = 804.31 MW
-------------------------------------------------------
----- Flue Gas Properties -----
Temperature: 670.0 °C (943.15 K)
Mixture Cp: 1.214 kJ/kg/K
-------------------------------------------------------
----- Energy Balance and Exhaust Flow -----
ΔT = 660.0 K
Exhaust Mass Flow = 1004.14 kg/s
Air-Fuel Ratio (AFR) = 33.40
-------------------------------------------------------


In [ ]:
from CoolProp.CoolProp import PropsSI

# Constants
P_ATM = 101325  # Pa
T_C = 15        # temperature in Celsius for cooling water
T_K = T_C + 273.15

# CoolProp incompressible fluid name for 50% ethylene glycol (mass fraction)
fluid = 'INCOMP::MEG-50%'

# Get specific heat (Cp) in J/kg/K
cp_mass = PropsSI("C", "T", T_K, "P", P_ATM, fluid)

# Get density in kg/m^3
density = PropsSI("Dmass", "T", T_K, "P", P_ATM, fluid)

# Get dynamic viscosity in Pa·s
mu = PropsSI("VISCOSITY", "T", T_K, "P", P_ATM, fluid)

print(f"50-50 Water/Ethylene Glycol (mass-based) specific heat at {T_C}°C: {cp_mass/1000:.4f} kJ/kg/K")
print(f"50-50 Water/Ethylene Glycol (mass-based) density at {T_C}°C: {density:.2f} kg/m³")
print(f"50-50 Water/Ethylene Glycol (mass-based) dynamic viscosity at {T_C}°C: {mu*1000:.4f} mPa·s")



50-50 Water/Ethylene Glycol (mass-based) specific heat at 15°C: 3.2855 kJ/kg/K
50-50 Water/Ethylene Glycol (mass-based) density at 15°C: 1067.53 kg/m³
50-50 Water/Ethylene Glycol (mass-based) dynamic viscosity at 15°C: 4.3763 mPa·s


In [ ]:
# Natural Gas CP calculation as a function of composition and temperature

from CoolProp.CoolProp import PropsSI

# --- Constants ---
P_ATM = 101325  # Pa (1 atm)
T_C = 78.62     # °C  <<== CHANGE THIS VALUE MANUALLY
T_K = T_C + 273.15

# --- Mixture Definition (Mole Fractions) ---

MOLE_FRACTIONS = {
    'CO2': 0.04925363,   # Carbon dioxide
    'SO2': 0.00000012,   # Sulfur dioxide (trace)
    'N2':  0.73794838,   # Nitrogen
    'H2O': 0.10024462,   # Water vapor
    'O2':  0.10389788,   # Oxygen
    'Ar':  0.00865537    # Argon
}

'''
#AIR COMPOSITION
MOLE_FRACTIONS = {
    'CO2': 0.0004,   # Carbon dioxide
    'SO2': 0,   # Sulfur dioxide (trace)
    'N2':  0.7808,   # Nitrogen
    'H2O': 0,   # Water vapor
    'O2':  0.2095,   # Oxygen
    'Ar':  0.0093    # Argon
}

'''
# --- Molar Masses (kg/mol) ---
MOLAR_MASSES_KG = {
    'CO2': 44.0095 / 1000.0,
    'SO2': 64.0660 / 1000.0,
    'N2':  28.0134 / 1000.0,
    'H2O': 18.01528 / 1000.0,
    'O2':  31.9988 / 1000.0,
    'Ar':  39.948 / 1000.0,
}

# --- Calculate Average Molar Mass (mixture) ---
MM_mix = sum(MOLE_FRACTIONS[comp] * MOLAR_MASSES_KG[comp] for comp in MOLE_FRACTIONS)

# --- Function: Get Cp for one component ---
def get_cp_component(comp, T_K):
    """Return Cp in J/mol/K and J/kg/K for a component at T_K and 1 atm."""
    cp_mass = PropsSI("CPMASS", "T", T_K, "P", P_ATM, comp)  # J/kg/K
    M_i = MOLAR_MASSES_KG[comp]
    cp_molar = cp_mass * M_i  # J/mol/K
    return cp_mass, cp_molar

# --- Function: Mixture Cp ---
def mixture_cp(T_K):
    """
    Returns mass-based cp (kJ/kg/K) for the mixture using CoolProp data.
    """
    cp_molar_mix = 0.0
    for comp, y_i in MOLE_FRACTIONS.items():
        _, cp_molar_i = get_cp_component(comp, T_K)
        cp_molar_mix += y_i * cp_molar_i

    # Convert to mass basis
    cp_mass_mix = cp_molar_mix / MM_mix  # J/kg/K
    return cp_mass_mix / 1000.0  # kJ/kg/K

# --- Main Execution ---
if __name__ == "__main__":
    print(f"--- Gas Mixture Cp Calculation ---")
    print(f"Temperature: {T_C:.2f} °C ({T_K:.2f} K)")
    print(f"Pressure: {P_ATM/1000:.2f} kPa")
    print("-" * 45)

    print("Component Cp values:")
    for comp in MOLE_FRACTIONS:
        cp_mass, _ = get_cp_component(comp, T_K)
        print(f"{comp:>4s}: {cp_mass/1000:.4f} kJ/kg/K")

    cp_mix = mixture_cp(T_K)
    print("-" * 45)
    print(f"Average Molar Mass: {MM_mix:.5f} kg/mol")
    print(f"Mixture Cp: {cp_mix:.4f} kJ/kg/K")
    print("-" * 45)


--- Gas Mixture Cp Calculation ---
Temperature: 78.62 °C (351.77 K)
Pressure: 101.33 kPa
---------------------------------------------
Component Cp values:
 CO2: 0.9007 kJ/kg/K
 SO2: 0.6649 kJ/kg/K
  N2: 1.0424 kJ/kg/K
 H2O: 4.1957 kJ/kg/K
  O2: 0.9295 kJ/kg/K
  Ar: 0.5212 kJ/kg/K
---------------------------------------------
Average Molar Mass: 0.02832 kg/mol
Mixture Cp: 1.2130 kJ/kg/K
---------------------------------------------


In [ ]:
# Natural Gas LHV Calculation (in kJ/kg)  as a function of composition

# Gas composition (mol-%)
composition = {
    "CH4": 89.138,
    "N2": 0.504,
    "CO2": 0.446,
    "C2H6": 5.691,
    "C3H8": 2.406,
    "nC4H10": 0.795,
    "iC4H10": 0.321,
    "nC5H12": 0.218,
    "iC5H12": 0.200,
    "nC6H14": 0.280,
    "H2S": 0.000288,
}

# Convert to mole fraction
total = sum(composition.values())
for k in composition:
    composition[k] /= total

# Standard LHVs (kJ/mol) — from NIST & Perry’s Handbook
LHV_data_kJ_per_mol = {
    "CH4": 802.3,
    "C2H6": 1428.0,
    "C3H8": 2043.0,
    "nC4H10": 2658.0,
    "iC4H10": 2658.0,
    "nC5H12": 3272.0,
    "iC5H12": 3268.0,
    "nC6H14": 3890.0,
    "CO2": 0.0,
    "N2": 0.0,
    "H2S": 563.0,  # approximate
}

# Molecular weights (g/mol) — from NIST Chemistry WebBook
MW_data = {
    "CH4": 16.043,
    "C2H6": 30.070,
    "C3H8": 44.097,
    "nC4H10": 58.124,
    "iC4H10": 58.124,
    "nC5H12": 72.151,
    "iC5H12": 72.151,
    "nC6H14": 86.178,
    "CO2": 44.010,
    "N2": 28.013,
    "H2S": 34.076,
}

# Calculate mixture molecular weight (g/mol)
MW_mix = sum(composition[comp] * MW_data[comp] for comp in composition)

# Calculate LHV mixture (kJ/mol)
LHV_mix_mol = sum(composition[comp] * LHV_data_kJ_per_mol.get(comp, 0) for comp in composition)

# Convert to kJ/kg
LHV_mix_mass = LHV_mix_mol / (MW_mix / 1000)  # (kJ/mol) / (kg/mol) = kJ/kg

# Display results
print("----- Natural Gas LHV Calculation -----")
print(f"Mixture Molecular Weight = {MW_mix:.3f} g/mol")
print(f"LHV (kJ/mol) = {LHV_mix_mol:.3f}")
print(f"LHV (kJ/kg)  = {LHV_mix_mass:.2f}")


----- Natural Gas LHV Calculation -----
Mixture Molecular Weight = 18.602 g/mol
LHV (kJ/mol) = 899.809
LHV (kJ/kg)  = 48371.81
